<h1 style='color:red; text-align:center;'>Vendor Network Optimization for City-wide Product Delivery</h1>

<h2>Introduction</h2>
<p>
This project aims to enhance the 
    <span style='font-weight:bold;'>
        efficiency of product delivery 
    </span>
    across a city by optimizing the 
    <span style='font-weight: bold;'>
    vendor network.
    </span>
        
    
</br>
    Our primary focus is to 
    <span style='color:red;'>
        maximize
    </span> 
    <span style='font-weight: bold;'>
        city-wide coverage
    </span>
    by strategically selecting vendors who offer the 
    <span style='font-weight:bold;'>
        best combination of 
        <span style='color:red;'>
            high performance
        </span>and 
        <span style='color:red;'>
            customer satisfaction
        </span>
    </span>.
    
</p>

<ul>
    <li>
        <span style='font-weight: bold;'>
            Vendor Network Optimization
        </span>
        <p>
            The objective is to build a network of vendors that maximizes customer coverage. Coverage is defined as the proportion of customer sessions within a 2 km radius of any vendor. 
            The key goals of this part include:
            <ul>
                <li>Maximizing the number of customer sessions covered by vendors.</li>
                <li>Prioritizing vendors with higher customer satisfaction ratings, based on performance scores.</li>
                <li>Ensuring that certain mandatory vendors are included while excluding blacklisted vendors.</li>
            </ul>
        </p>
    </li>
</ul>
</br>
<p>
    <span style='font-weight: bold; color:red;'>
        Note:
    </span>
    Based on company rules, I can't use actual data, so I create sample datas and run code for them.
</p>

In [44]:
!pip install pandas numpy

In [38]:
# read and analize data
import numpy as np
import pandas as pd

In [42]:
# read data
vendors = pd.read_csv('DataSets/sample_vendors.csv')
sessions = pd.read_csv('DataSets/sample_sessions.csv')